Founders and Financials: Machine Learning Algorithms in Venture Capital by Viktor Lado Naess and Emrik Stål



In [ ]:
## getting system info

import sys

print("Python version")
print(sys.version)
print("Version info.")
print(sys.version_info)

#START


In [ ]:
# import our google sheet that was the data from the end of stata
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

spreadsheet = gc.open('Unclean_data')

worksheet = spreadsheet.worksheet('Sheet1')


data = worksheet.get_all_values()

import pandas as pd
Original_df = pd.DataFrame(data[1:], columns=data[0])
Original_df


In [ ]:
#renmaming columns to more intuitive names
Original_df.rename(columns={'rr07_rorreul': 'Operating_profit'}, inplace=True)
Original_df.rename(columns={'rr12_resefin': 'Profit_after_net_Financial_Income/expenses'}, inplace=True)
Original_df.rename(columns={'rr15_resar': 'Net_profit_for_year'}, inplace=True)
Original_df.rename(columns={'ser_nystartat': 'Founed_this_year'}, inplace=True)
Original_df.rename(columns={'rr01_ntoms': 'Net_sales'}, inplace=True)
Original_df.rename(columns={'ny_kapomsh': 'Asset_turnover_ratio'}, inplace=True)
Original_df.rename(columns={'ny_avtokap': 'Return_on_assets'}, inplace=True)
Original_df.rename(columns={'ny_solid': 'Equity/asset_ratio'}, inplace=True)
Original_df.rename(columns={'ny_avkegkap': 'Return_on_equity'}, inplace=True)
Original_df.rename(columns={'ny_kassalikv': 'Quick_ratio'}, inplace=True)
Original_df.rename(columns={'ny_rormarg': 'Operating_margin'}, inplace=True)
Original_df.rename(columns={'ny_nettomarg': 'Net_margin'}, inplace=True)
Original_df.rename(columns={'ny_vinstprc': 'Profit_margin'}, inplace=True)
Original_df.rename(columns={'ny_omsf': 'Change_Net_sales'}, inplace=True)
Original_df.rename(columns={'bransch_borsbransch_konv': 'Branch_sector'}, inplace=True)
Original_df.rename(columns={'ser_regdat': 'Reg_Date'}, inplace=True)
Original_df.rename(columns={'regYear ': 'Reg_year'}, inplace=True)
Original_df.columns = [col.strip() for col in Original_df.columns]
import numpy as np
#replacing empty values with these columns with 0
columns_to_replace_with_zero = ['IPO', 'Merger_Acqusition', 'Serial_Founder']
for column in columns_to_replace_with_zero:
    Original_df[column].replace('', np.nan, inplace=True)
    Original_df[column].replace(np.nan, 0, inplace=True)
    Original_df[column] = Original_df[column].astype(int)
#replacing empty sections with namn
for column in Original_df.columns:
    Original_df[column].replace('', np.nan, inplace=True)
#chaning Name column to a string
Original_df.loc[:,'Name'] = Original_df['Name'].astype(str)
Original_df['Name'] = Original_df['Name'].str.strip()

In the following section we remove any rows that have too little data to be useful, intially we remove rows with non of the three financial variables but ultimately we remove all rows that are missing one of the three financial varibales as after some investigaiton there are few rows to gain if one were to implement kmeans. The investigation can be done by uncommenting diffent lines of code in the second chunk after this section.

In [ ]:

import pandas as pd
triple_na_rows = Original_df[Original_df['Quick_ratio'].isna() & Original_df['Return_on_equity'].isna() & Original_df['Net_sales'].isna()]


Original_df = Original_df.drop(triple_na_rows.index)
Original_df

In [ ]:

#Quick_ratio X Net_sales
# na_rows = Original_df[Original_df['Quick_ratio'].isna() & Original_df['Net_sales'].isna()]

#Return_on_equity X Net_sales
# na_rows = Original_df[Original_df['Net_sales'].isna() & Original_df['Return_on_equity'].isna()]

# na_rows = Original_df[Original_df['Quick_ratio'].isna()]
# big_3_na_rows = Original_df[Original_df['Quick_ratio'].isna() | Original_df['Return_on_equity'].isna() | Original_df['Net_sales'].isna() | Original_df['Branch_sector'].isna()]

big_3_na_rows = Original_df[Original_df['Quick_ratio'].isna() | Original_df['Return_on_equity'].isna() | Original_df['Net_sales'].isna()]
# big_3_na_rows
Original_df = Original_df.drop(big_3_na_rows.index)
Original_df

In [ ]:
feature_columns = ['Quick_ratio', 'Return_on_equity', 'Net_sales']
for column in feature_columns:
  Original_df[column] = Original_df[column].replace('', np.nan)
  Original_df = Original_df.dropna(subset=[column])
  Original_df.loc[:,column] = Original_df[column].astype(float)


In [ ]:
Original_df


In [ ]:
Original_df.dtypes #checking datatypes of columns

#DATA MAPPING

In [ ]:
from os import name

#columns are made the appropirate data type
df= Original_df.copy()
df = df.drop(['ser_year', 'ser_namn','Search_Year'], axis=1)
df.loc[:,'Name'] = df['Name'].astype(str)
df.loc[:,'Location'] = df['Location'].astype(str)
df.loc[:,'Founded_on_Year'] = df['Founded_on_Year'].astype(int)
df.loc[:,'More_than_one_round'] = df['More_than_one_round'].astype(int)
df.loc[:,'Number_of_rounds'] = df['Number_of_rounds'].astype(int)
df.loc[:,'Merger_Acqusition'] = df['Merger_Acqusition'].astype(int)



In [ ]:
import pandas as pd #In the following, nan values in IPO and Merger_Acqusition are made 0. This is because we left when making these columns we empty cells represent no IPO Merger/acquisition respectively
import numpy as np
int_columns = ['IPO', 'Merger_Acqusition']
for column in int_columns:
  df[column] = pd.to_numeric(df[column], errors='coerce')
  df[column] = df[column].replace('', np.nan)
  df[column] = df[column].fillna(0)
  df[column] = df[column].astype(int)


In [ ]:
#here we drop rows with no branch sector or final second round year
int_columns = ['Branch_sector', 'Final_Second_Round_Year']
for column in int_columns:
  df[column] = pd.to_numeric(df[column], errors='coerce')
  df[column] = df[column].replace('', np.nan)
  df = df.dropna(subset=['Branch_sector'])
  df[column] = df[column].astype(int)
df

In [ ]:
#here we drop rows with no Quick_ratio,Return_on_equity, Net_sales but more importantly we make these columns float variables
feature_columns = ['Quick_ratio', 'Return_on_equity', 'Net_sales']
for column in feature_columns:
  df[column] = df[column].replace('', np.nan)
  df = df.dropna(subset=[column])
  df.loc[:,column] = df[column].astype(float)

In [ ]:
cols = ['Name','More_than_one_round', 'Founded_on_Year', 'Final_Second_Round_Year', 'Distance_from_Stockholm', 'Serial_Founder', 'Quick_ratio', 'Return_on_equity', 'Net_sales', 'Branch_sector', 'Share female founders', 'Location']
df=df[cols]

In [ ]:
df


In [ ]:
df.loc[:,'Return_on_equity'] = df['Return_on_equity'].astype(float)
df.loc[:,'Net_sales'] = df['Net_sales'].astype(float)
df.loc[:,'Quick_ratio'] = df['Quick_ratio'].astype(float)

In [ ]:
df.dtypes


In [ ]:
df['Location'] = df['Location'].str.strip()

In [ ]:
df

#adding benchmarked net sales

In [ ]:
#Here we implement the net sales benchmark, by importing sheet2 form Unclean_data work sheet
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

spreadsheet = gc.open('Unclean_data')

worksheet = spreadsheet.worksheet('Sheet2')
#worksheet = spreadsheet.get_worksheet(0)

# List all worksheets in the spreadsheet
# all_worksheets = spreadsheet.worksheets()
# print([ws.title for ws in all_worksheets])


data = worksheet.get_all_values()

import pandas as pd
NS_BM = pd.DataFrame(data[1:], columns=data[0])
NS_BM



In [ ]:
df

In [ ]:
# making sure there is name consistency and data type consistency prior to merger
NS_BM.rename(columns={'Year': 'Final_Second_Round_Year'}, inplace=True)
NS_BM.rename(columns={'Branch': 'Branch_sector'}, inplace=True)
NS_BM.rename(columns={'regYear': 'Founded_on_Year'}, inplace=True)
NS_BM.loc[:,'Founded_on_Year'] = NS_BM['Founded_on_Year'].astype(int)

int_columns = ['Branch_sector', 'Final_Second_Round_Year']
for column in int_columns:
  NS_BM[column] = pd.to_numeric(NS_BM[column], errors='coerce')
  NS_BM[column] = NS_BM[column].replace('', np.nan)

  NS_BM = NS_BM.dropna(subset=['Branch_sector'])
  NS_BM[column] = NS_BM[column].astype(int)

NS_BM.loc[:,'avg'] = NS_BM['avg'].astype(float)


NS_BM

In [ ]:
df

In [ ]:
# Perform the merge with an outer join
merged_df = pd.merge(df, NS_BM[['Founded_on_Year', 'Final_Second_Round_Year', 'Branch_sector', 'avg']],
                     on=['Founded_on_Year', 'Final_Second_Round_Year', 'Branch_sector'],
                     how='outer', indicator=True)

# Filter to find rows that were in the original data frame but not in the new import one
dropped_rows = merged_df[merged_df['_merge'] == 'left_only']


dropped_rows


In [ ]:
df = pd.merge(df, NS_BM[['Founded_on_Year', 'Final_Second_Round_Year', 'Branch_sector', 'avg']],
                     on=['Founded_on_Year', 'Final_Second_Round_Year', 'Branch_sector'],
                     how='inner')

In [ ]:
df

In [ ]:
#making net sales ratio
df['net_sale_ratio'] = (df['Net_sales'] / df['avg'])


In [ ]:
# making sure that the two columns are numeric
df['Distance_from_Stockholm'] = pd.to_numeric(df['Distance_from_Stockholm'], errors='coerce')
df['Share female founders'] = pd.to_numeric(df['Share female founders'], errors='coerce')

In [ ]:
#some finally renaming as to make graphs nicer
Namecols = ['Name','More_than_one_round','Share female founders', 'Distance_from_Stockholm', 'Location', 'Serial_Founder', 'Quick_ratio', 'Return_on_equity', 'Net_sales', 'net_sale_ratio']
FinNames=df[Namecols]
cols = ['More_than_one_round','Share female founders', 'Distance_from_Stockholm', 'Location', 'Serial_Founder', 'Quick_ratio', 'Return_on_equity', 'Net_sales', 'net_sale_ratio' ,'Final_Second_Round_Year', 'Founded_on_Year', 'Branch_sector','avg']

df=df[cols]



df.rename(columns={'More_than_one_round': 'More than one round'}, inplace=True)
df.rename(columns={'Distance_from_Stockholm': 'Distance from Stockholm'}, inplace=True)
df.rename(columns={'Serial_Founder': 'Serial Founder'}, inplace=True)
df.rename(columns={'Quick_ratio': 'Quick Ratio'}, inplace=True)
df.rename(columns={'Return_on_equity': 'Return on Equity'}, inplace=True)
df.rename(columns={'net_sale_ratio': 'Net Sales Ratio'}, inplace=True)
df.rename(columns={'Branch_sector': 'Branch Sector'}, inplace=True)

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.expand_frame_repr = False
pd.options.display.max_colwidth = -1
df

In [ ]:
# # The below code will make a google sheet with the above dataframe this the final data, that is used in the following sections. However as noted in the README the accuarcy of the data here is rounded to a higher extent. So if this is run
## be sure to edit the sheet, specifically set cell i2= h2/m2 and then drag down until row 219.

# df.replace([np.nan, np.inf, -np.inf], None, inplace=True)
# from google.colab import auth
# auth.authenticate_user()

# import gspread
# from google.auth import default
# creds, _ = default()
# gc = gspread.authorize(creds)

# sh = gc.create('final data')
# worksheet = sh.get_worksheet(0)
# worksheet.clear()
# worksheet.append_row(df.columns.tolist())
# for index, row in df.iterrows():
#     worksheet.append_row(row.tolist())

# headers = df.columns.tolist()